В целом этот код сгенерил GPT.
В нем реализован метод Рунге Кутты 4 порядка, используется разностная аппроксимация для производной по координате. 
В том числе из-за плохо подходящей для данной задачи разностной аппроксимации, в решениях наблюдаются сильные расхождения.

В программе также используются формулы для перехода к размерным величинам (в системе СИ) 

In [1]:
import numpy as np
import matplotlib.pyplot as plt

### Cферически симметричный случай

In [ ]:
gamma = 1.4 
rho0 = 1.225  # кг/м^3 (плотность воздуха на уровне моря)
p0   = 101325.0  # Па (атмосферное давление)
L0   = 1.0     # м (характерная длина)
U0   = np.sqrt(gamma * p0 / rho0) # м/с (характерная скорость, примерно скорость звука)
T0 = p0 / (rho0 * 287.0) # K (Характерная температура)
t_0   = L0 / U0    # Характерное время (с)

def Conversion_in_Dim_quantities(x, rho, u, p, T):
    return x*L0, rho*rho0, u*U0, p*p0, T*T0

def Conversion_in_Dim_less_quantities(x, rho, u, p, T):
    return x/L0, rho/rho0, u/U0, p/p0, T/T0

def ConversionTime_in_Dim(t):
    return t*t_0

def ConversionTime_in_Dim_less(t):
    return t/t_0
print(rho0,p0,L0,U0,T0, t_0, sep='\n')

1.225
101325.0
1.0
340.29399054347107
288.20308611249374
0.002938635496921167


In [40]:
def Initial_Conditional(r_bound, r : list):
    rho = 0.1*np.ones(n_points) + 0.5*np.exp(-(r-r_bound)**2/0.01**2)
    u = np.zeros(n_points)
    p = np.ones(n_points)
    T = np.ones(n_points)
    
    #rho[r >= r_bound] = 0.125
    #u[r >= r_bound] = 0.0
    #p[r >= r_bound] = 0.1
    T = p/rho/287
    
    return rho, u, p, T

def calculate_fluxes(rho, u, p, gamma, r): #Добавляем r в аргументы
    F_rho  = rho * u
    F_rhou = rho * u**2 + p
    F_rhoE    = (0.5*rho*u**2 + p/(gamma - 1))*u*rho
    return F_rho, F_rhou, F_rhoE

# --------------------- Функция обновления переменных (RK4) ---------------------
def rk4_step(rho, u, p, r, dr, dt, gamma): # Добавляем r в аргументы
    E = p / (gamma - 1) + 0.5 * rho * u**2
    F_rho, F_rhou, F_rhoE = calculate_fluxes(rho, u, p, gamma, r) #Добавляем r

    # Сферическая геометрия:  (1/r^2) * d/dr (r^2 * flux)
    # Мы используем центральные разности для d/dr и используем средние значения r^2

    # Функция для расчета r^2 * flux в узлах сетки
    def r_squared_flux(r, flux):
        return r**2 * flux

    # k1
    r2F_rho = r_squared_flux(r, F_rho)
    r2F_rhou = r_squared_flux(r, F_rhou)
    r2F_rhoE = r_squared_flux(r, F_rhoE)

    #Вычисляем производную используя центральную разность
    drho_dt1  = - (np.roll(r2F_rho, -1) - np.roll(r2F_rho, 1)) / (2 * dr) / r**2 #Добавляем /r**2
    drhou_dt1 = - (np.roll(r2F_rhou, -1) - np.roll(r2F_rhou, 1)) / (2 * dr) / r**2 + 2*p/r  #Добавляем /r**2  + 2p/r
    dE_dt1    = - (np.roll(r2F_rhoE, -1) - np.roll(r2F_rhoE, 1)) / (2 * dr) / r**2 #Добавляем /r**2

    k1_rho  = dt * drho_dt1
    k1_rhou = dt * drhou_dt1
    k1_E    = dt * dE_dt1

    # k2
    rho_temp = rho + k1_rho / 2
    u_temp   = u + k1_rhou / (2*rho) / 2
    E_temp = E + k1_E / 2
    p_temp = (gamma - 1)*(E_temp - 0.5*rho_temp*u_temp**2)
    F_rho, F_rhou, F_rhoE = calculate_fluxes(rho_temp, u_temp, p_temp, gamma, r) #Добавляем r

    r2F_rho = r_squared_flux(r, F_rho)
    r2F_rhou = r_squared_flux(r, F_rhou)
    r2F_rhoE = r_squared_flux(r, F_rhoE)

    drho_dt2  = - (np.roll(r2F_rho, -1) - np.roll(r2F_rho, 1)) / (2 * dr) / r**2  #Добавляем /r**2
    drhou_dt2 = - (np.roll(r2F_rhou, -1) - np.roll(r2F_rhou, 1)) / (2 * dr) / r**2  + 2*p_temp/r#Добавляем /r**2  + 2p/r
    dE_dt2    = - (np.roll(r2F_rhoE, -1) - np.roll(r2F_rhoE, 1)) / (2 * dr) / r**2#Добавляем /r**2

    k2_rho  = dt * drho_dt2
    k2_rhou = dt * drhou_dt2
    k2_E    = dt * dE_dt2

    # k3
    rho_temp = rho + k2_rho / 2
    u_temp   = u + k2_rhou / (2*rho) / 2
    E_temp = E + k2_E / 2
    p_temp = (gamma - 1)*(E_temp - 0.5*rho_temp*u_temp**2)
    F_rho, F_rhou, F_rhoE = calculate_fluxes(rho_temp, u_temp, p_temp, gamma, r)#Добавляем r

    r2F_rho = r_squared_flux(r, F_rho)
    r2F_rhou = r_squared_flux(r, F_rhou)
    r2F_rhoE = r_squared_flux(r, F_rhoE)

    drho_dt3  = - (np.roll(r2F_rho, -1) - np.roll(r2F_rho, 1)) / (2 * dr) / r**2 #Добавляем /r**2
    drhou_dt3 = - (np.roll(r2F_rhou, -1) - np.roll(r2F_rhou, 1)) / (2 * dr) / r**2  + 2*p_temp/r#Добавляем /r**2  + 2p/r
    dE_dt3    = - (np.roll(r2F_rhoE, -1) - np.roll(r2F_rhoE, 1)) / (2 * dr) / r**2#Добавляем /r**2

    k3_rho  = dt * drho_dt3
    k3_rhou = dt * drhou_dt3
    k3_E    = dt * dE_dt3

    # k4
    rho_temp = rho + k3_rho
    u_temp   = u + k3_rhou / (2*rho)
    E_temp = E + k3_E
    p_temp = (gamma - 1)*(E_temp - 0.5*rho_temp*u_temp**2)
    F_rho, F_rhou, F_rhoE = calculate_fluxes(rho_temp, u_temp, p_temp, gamma,r)#Добавляем r

    r2F_rho = r_squared_flux(r, F_rho)
    r2F_rhou = r_squared_flux(r, F_rhou)
    r2F_rhoE = r_squared_flux(r, F_rhoE)

    drho_dt4  = - (np.roll(r2F_rho, -1) - np.roll(r2F_rho, 1)) / (2 * dr) / r**2#Добавляем /r**2
    drhou_dt4 = - (np.roll(r2F_rhou, -1) - np.roll(r2F_rhou, 1)) / (2 * dr) / r**2 + 2*p_temp/r #Добавляем /r**2  + 2p/r
    dE_dt4    = - (np.roll(r2F_rhoE, -1) - np.roll(r2F_rhoE, 1)) / (2 * dr) / r**2#Добавляем /r**2

    k4_rho  = dt * drho_dt4
    k4_rhou = dt * drhou_dt4
    k4_E    = dt * dE_dt4

    # Обновление переменных
    rho = rho + (k1_rho + 2*k2_rho + 2*k3_rho + k4_rho) / 6
    u   = u   + (k1_rhou + 2*k2_rhou + 2*k3_rhou + k4_rhou) / 6
    E   = E   + (k1_E + 2*k2_E + 2*k3_E + k4_E) / 6
    p = (gamma - 1)*(E - 0.5*rho*u**2)
    
    rho[0] = rho[1]
    u[0] = -u[1]  # Отражаем скорость (симметрия)
    p[0] = p[1]

    rho[-1] = rho[-2] + (rho[-2] - rho[-3])
    u[-1] = u[-2] + (u[-2] - u[-3])
    p[-1] = p[-2] + (p[-2] - p[-3])
    return rho, u, p

# ----------- Инициализация сетки и переменных (сферическая геометрия) -----------
n_points = 500 # Number of grid points
r_min = 0.1    # Minimum radius (avoiding r=0 exactly)
r_max = 1.0    # Maximum radius
r = np.linspace(r_min, r_max, n_points)  # Радиальная координата
dr = r[1] - r[0]  # Шаг по радиусу

# -----------  Инициализация переменных (пример) -----------


gamma = 1.4                  # Показатель адиабаты

#r, rho, u, p, T = r, rho_prime, u_prime, p_prime, T_prime
rho, u, p, T = Initial_Conditional(r[0], r)

# ----------- Временные параметры -----------
t      = 0.0
t_end_dim  = 0.003 #  В секундах
cfl    = 0.4     # CFL number (adjust as needed)
dt_dim     = 1.0e-2 # Initial time step (adjust as needed)
t_end, dt = ConversionTime_in_Dim_less(np.array([t_end_dim, dt_dim]))
print(t_end, dt)
#  ----------- Основной цикл времени -----------
while t < t_end:
    # Ограничение шага по времени (теперь с размерной скоростью звука)
    a = np.sqrt(gamma * p / rho)  # скорость звука
    #print(dt,cfl,dr,a,np.max(np.abs(u) + a))
    print(np.max(a))
    dt = min(dt, cfl * dr / np.max(np.abs(u) + a))  # Шаг по времени
    #, max(p), max(a*U0))
    
    rho, u, p = rk4_step(rho, u, p, r, dr, dt, gamma)  # Передаем r и dr
    t += dt
    print(f"Time: {ConversionTime_in_Dim(t):.6f}")


#print(rho)
# Общие настройки для графиков
fig_size = (6, 4) 
line_color = 'black'
grid_linestyle = '--'
grid_alpha = 0.5
grid_color = 'gray'

def create_plot(x_data, y_data, x_label, y_label, title):
    plt.figure(figsize=fig_size) 
    plt.plot(x_data, y_data, color=line_color)
    plt.xlabel(x_label, fontsize=12)
    plt.ylabel(y_label, fontsize=12)
    plt.title(title, fontsize=14)
    plt.grid(True, linestyle=grid_linestyle, alpha=grid_alpha, color=grid_color)
    # plt.ylim(0,2) # Пример: установить предел по оси Y от 0 до 2
    plt.show()
    plt.close()

r = np.linspace(r_min, r_max, n_points)
#r, rho, u, p, T = Conversion_in_Dim_quantities(r, rho, u, p, T)
# Density plot
create_plot(r, rho, 'r (м)', 'Плотность (кг/м^3)', 'Плотность')
#del x, rho

# Velocity plot
create_plot(r, u, 'r (м)', 'Скорость (м/с)', 'Скорость')
#del x, u

# Pressure plot
#x = np.linspace(xmin, xmax, nx)
create_plot(r, p, 'x (м)', 'Давление (Па)', 'Давление')
#del x, p
# Temperature plot
#x = np.linspace(xmin, xmax, nx)
create_plot(r, T, 'x (м)', 'Температура (К)', 'Температура')
#del x, T

1.0208819716304132 3.4029399054347107
3.7416573867739413
Time: 0.000001
3.765429703253376
Time: 0.000001
3.7894354680065887
Time: 0.000002
3.813662123970801
Time: 0.000002
3.8380953691351762
Time: 0.000003
3.8627278498367215
Time: 0.000003
3.887552256048794
Time: 0.000004
3.912563446026144
Time: 0.000004
3.9377599287121248
Time: 0.000005
3.9631412918790327
Time: 0.000006
3.9887078187457714
Time: 0.000006
4.014460232423281
Time: 0.000007
4.04039951434143
Time: 0.000007
4.066526761324357
Time: 0.000008
4.092843057714132
Time: 0.000008
4.119349347840249
Time: 0.000009
4.146046302248474
Time: 0.000009
4.172934179116969
Time: 0.000010
4.200012689711275
Time: 0.000010
4.227280882193296
Time: 0.000011
4.254737059932934
Time: 0.000011
4.282378747545974
Time: 0.000012
4.310202710400713
Time: 0.000012
4.338205023133443
Time: 0.000013
4.366381172788429
Time: 0.000013
4.394726175622614
Time: 0.000014
4.4232346853440445
Time: 0.000014
4.451901074741764
Time: 0.000015
4.480719480764161
Time: 0.00001

C:\Users\izavo\AppData\Local\Temp\ipykernel_21932\4126292602.py:143: RuntimeWarning: invalid value encountered in sqrt
  a = np.sqrt(gamma * p / rho)  # скорость звука


Time: 0.000236
nan
Time: 0.000236
nan
Time: 0.000236
nan
Time: 0.000236
nan
Time: 0.000236
nan
Time: 0.000236
nan
Time: 0.000236
nan
Time: 0.000236
nan
Time: 0.000236
nan
Time: 0.000236
nan
Time: 0.000236
nan
Time: 0.000236
nan
Time: 0.000236
nan
Time: 0.000236
nan
Time: 0.000236
nan
Time: 0.000236
nan
Time: 0.000236
nan
Time: 0.000236
nan
Time: 0.000236
nan
Time: 0.000236
nan
Time: 0.000236
nan
Time: 0.000236
nan
Time: 0.000236
nan
Time: 0.000236
nan
Time: 0.000236
nan
Time: 0.000236
nan
Time: 0.000236
nan
Time: 0.000236
nan
Time: 0.000236
nan
Time: 0.000236
nan
Time: 0.000236
nan
Time: 0.000237
nan
Time: 0.000237
nan
Time: 0.000237
nan
Time: 0.000237
nan
Time: 0.000237
nan
Time: 0.000237
nan
Time: 0.000237
nan
Time: 0.000237
nan
Time: 0.000237
nan
Time: 0.000237
nan
Time: 0.000237
nan
Time: 0.000237
nan
Time: 0.000237
nan
Time: 0.000237
nan
Time: 0.000237
nan
Time: 0.000237
nan
Time: 0.000237
nan
Time: 0.000237
nan
Time: 0.000237
nan
Time: 0.000237
nan
Time: 0.000237
nan
Time: 0.0002

C:\Users\izavo\AppData\Local\Temp\ipykernel_21932\4126292602.py:17: RuntimeWarning: overflow encountered in multiply
  F_rhoE    = (0.5*rho*u**2 + p/(gamma - 1))*u*rho
C:\Users\izavo\AppData\Local\Temp\ipykernel_21932\4126292602.py:88: RuntimeWarning: overflow encountered in square
  p_temp = (gamma - 1)*(E_temp - 0.5*rho_temp*u_temp**2)
C:\Users\izavo\AppData\Local\Temp\ipykernel_21932\4126292602.py:88: RuntimeWarning: invalid value encountered in subtract
  p_temp = (gamma - 1)*(E_temp - 0.5*rho_temp*u_temp**2)
C:\Users\izavo\AppData\Local\Temp\ipykernel_21932\4126292602.py:15: RuntimeWarning: overflow encountered in multiply
  F_rho  = rho * u
C:\Users\izavo\AppData\Local\Temp\ipykernel_21932\4126292602.py:16: RuntimeWarning: overflow encountered in square
  F_rhou = rho * u**2 + p
C:\Users\izavo\AppData\Local\Temp\ipykernel_21932\4126292602.py:16: RuntimeWarning: invalid value encountered in add
  F_rhou = rho * u**2 + p
C:\Users\izavo\AppData\Local\Temp\ipykernel_21932\4126292602.

Time: 0.000368
nan
Time: 0.000368
nan
Time: 0.000368
nan
Time: 0.000368
nan
Time: 0.000368
nan
Time: 0.000368
nan
Time: 0.000368
nan
Time: 0.000368
nan
Time: 0.000368
nan
Time: 0.000368
nan
Time: 0.000368
nan
Time: 0.000368
nan
Time: 0.000368
nan
Time: 0.000368
nan
Time: 0.000368
nan
Time: 0.000368
nan
Time: 0.000368
nan
Time: 0.000368
nan
Time: 0.000368
nan
Time: 0.000368
nan
Time: 0.000368
nan
Time: 0.000368
nan
Time: 0.000368
nan
Time: 0.000368
nan
Time: 0.000368
nan
Time: 0.000368
nan
Time: 0.000368
nan
Time: 0.000368
nan
Time: 0.000368
nan
Time: 0.000368
nan
Time: 0.000368
nan
Time: 0.000368
nan
Time: 0.000368
nan
Time: 0.000368
nan
Time: 0.000368
nan
Time: 0.000368
nan
Time: 0.000368
nan
Time: 0.000368
nan
Time: 0.000368
nan
Time: 0.000368
nan
Time: 0.000368
nan
Time: 0.000368
nan
Time: 0.000368
nan
Time: 0.000368
nan
Time: 0.000368
nan
Time: 0.000368
nan
Time: 0.000368
nan
Time: 0.000368
nan
Time: 0.000368
nan
Time: 0.000368
nan
Time: 0.000369
nan
Time: 0.000369
nan
Time: 0.0003

KeyboardInterrupt: 

##### Некоторые тесты сферически симметричного случая

1) Начальное условие: возмущение в плотности, должна быть скорость звука
ρ(r, 0) = ρ₀ + A * exp(-(r - r₀)² / σ²) (ρ₀ - средняя плотность, A - амплитуда возмущения, r₀ - положение центра возмущения, σ - ширина возмущения)
u(r, 0) = 0
p(r, 0) = p₀ (постоянное давление)